In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import ndimage
from tqdm import tqdm

h = 0.7
physical_size = 200  # Mpc/h

def calculateentire(plane='xy', sliceid=128, delta_z=5, loadnpy='', number="0050"):
    bubbles = np.load(loadnpy)
    grid_size = bubbles.shape[0]  # Assume square grid
    delta = physical_size / grid_size  # 0.78125 Mpc/h per grid cell
    z_min = sliceid - delta_z
    z_max = sliceid + delta_z
    bubblesid = sliceid * grid_size // 200

    if plane == 'xy':
        slice_data_cropped = bubbles[:, :, bubblesid]
        xstring, ystring, zstring = 'x', 'y', 'z'
    elif plane == 'yz':
        slice_data_cropped = bubbles[bubblesid, :, :]
        xstring, ystring, zstring = 'y', 'z', 'x'
    elif plane == 'zx':
        slice_data_cropped = np.transpose(bubbles[:, bubblesid, :])
        xstring, ystring, zstring = 'z', 'x', 'y'

    xfrac_2d = slice_data_cropped.copy()

    # Load the halo data
    file_path = f'jwst_data/CDM_200Mpc_2048/haloes/CDM_200Mpc_2048.0{number}.fof.txt'
    data = np.loadtxt(file_path)

    # Extract positions and masses
    if plane == 'xy':
        x_coords = data[:, 2] + 100
        y_coords = data[:, 1] + 100
        z_coords = data[:, 3] + 100
    elif plane == 'yz':
        x_coords = data[:, 3] + 100
        y_coords = data[:, 2] + 100
        z_coords = data[:, 1] + 100
    elif plane == 'zx':
        x_coords = data[:, 1] + 100
        y_coords = data[:, 3] + 100
        z_coords = data[:, 2] + 100

    masses = data[:, 0]  # Extract masses

    # Shift coordinates if necessary
    x_coords -= delta
    y_coords -= delta

    # Filter by z-range
    slice_mask = (z_coords >= z_min) & (z_coords <= z_max)
    x_slice = x_coords[slice_mask]
    y_slice = y_coords[slice_mask]
    mass_slice = masses[slice_mask]  # Filter the corresponding masses

    # Create an empty mass density grid
    halo_density_2d = np.zeros((grid_size, grid_size))

    # Convert coordinates to pixel indices
    x_indices = ((x_slice / physical_size) * grid_size).astype(int)
    y_indices = ((y_slice / physical_size) * grid_size).astype(int)

    # Ensure indices are within bounds
    valid_mask = (x_indices >= 0) & (x_indices < grid_size) & (y_indices >= 0) & (y_indices < grid_size)
    x_indices = x_indices[valid_mask]
    y_indices = y_indices[valid_mask]
    mass_slice = mass_slice[valid_mask]

    # Accumulate mass into each grid pixel
    np.add.at(halo_density_2d, (y_indices, x_indices), mass_slice)

    return xfrac_2d, halo_density_2d, zstring

redshift_input = int(input("At which redshift? [8: z=8.895/ 9: z=9.304/ 10: z=9.922]"))
sink = input("Which sink? [A/B/C]")
if redshift_input == 8:
    redshift, number, shortred = "8.895", "0050", "08"
elif redshift_input == 9:
    redshift, number, shortred = "9.304", "0047", "09"
elif redshift_input == 10:
    redshift, number, shortred = "9.922", "0043", "10"

delta_z = 5
no_of_bubbles = 20

file_xfrac = f'jwst_data/mini_200Mpc_Nion_4_Source1_Sink{sink.title()}/xfrac_{redshift}.npy'

for plane in ['xy']:
    output_dir = f'mock_data'
    os.makedirs(output_dir, exist_ok=True)
    isolated_bubbles_dir = os.path.join(output_dir, 'isolated_bubbles')
    halo_density_dir = os.path.join(output_dir, 'halo_density_patches')
    plots_dir = os.path.join(output_dir, 'plots')

    os.makedirs(isolated_bubbles_dir, exist_ok=True)
    os.makedirs(halo_density_dir, exist_ok=True)
    os.makedirs(plots_dir, exist_ok=True)

    for sliceid in tqdm(range(10, 191), desc=f"Processing slices from plane {plane}"):
        xfrac_2d, halo_density_2d, z_string = calculateentire(plane, sliceid, delta_z, file_xfrac, number)

        threshold = 0.5
        binary_array = xfrac_2d > threshold
        structure = np.ones((3, 3), dtype=int)
        labeled_array, num_features = ndimage.label(binary_array, structure=structure)

        no_of_bubbles_slice = min(no_of_bubbles, num_features)
        sizes = ndimage.sum(binary_array, labeled_array, index=np.arange(1, num_features + 1))
        sorted_indices = np.argsort(sizes)[::-1]
        largest_labels = sorted_indices[:no_of_bubbles_slice] + 1
        slices = ndimage.find_objects(labeled_array)

        patch_size = 64
        max_shape = xfrac_2d.shape

        for i, label in enumerate(largest_labels, start=1):
            bubble_mask = (labeled_array == label)
            original_slice = slices[label - 1]

            coords = np.argwhere(bubble_mask)
            y_coords, x_coords = coords[:, 0], coords[:, 1]
            y_center, x_center = np.mean(y_coords), np.mean(x_coords)

            half_patch = patch_size // 2
            y_start, x_start = int(round(y_center - half_patch)), int(round(x_center - half_patch))
            y_end, x_end = y_start + patch_size, x_start + patch_size

            y_indices = np.arange(y_start, y_end) % max_shape[0]
            x_indices = np.arange(x_start, x_end) % max_shape[1]

            data_patch = xfrac_2d[np.ix_(y_indices, x_indices)]
            halo_density_patch = halo_density_2d[np.ix_(y_indices, x_indices)]
            bubble_mask_patch = bubble_mask[np.ix_(y_indices, x_indices)]

            isolated_bubble = np.where(bubble_mask_patch, data_patch, 0)
            halo_density_filtered = np.where(bubble_mask_patch, halo_density_patch, 0)

            isolated_bubble_filename = os.path.join(isolated_bubbles_dir, f'slice_{sliceid}_bubble_{i}_isolated.npy')
            halo_density_filename = os.path.join(halo_density_dir, f'slice_{sliceid}_bubble_{i}_halo_density_filtered.npy')

            np.save(isolated_bubble_filename, isolated_bubble)
            np.save(halo_density_filename, halo_density_filtered)
    print(f"Saved isolated bubble patches and halo density patches for slices 10-191 in '{output_dir}' directory.")

Processing slices from plane xy:   1%|          | 1/181 [00:06<18:13,  6.08s/it]